In [20]:
# ! pip install pygam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.0/522.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 13.5 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1


In [21]:
from pygam import LinearGAM, s, f, l
import pandas as pd
import patsy as pt
import numpy as np
from plotly import subplots
import plotly.offline as py
import plotly.graph_objs as go

In [22]:
data = pd.read_csv("https://github.com/dustywhite7/econ8310-assignment1/raw/main/assignment_data_train.csv")

In [24]:
data['Timestamp'] = pd.to_datetime(data['Timestamp'],
	format='%Y-%m-%d %H:%M:%S')

In [25]:
data.set_index(pd.DatetimeIndex(data['Timestamp']),
	inplace=True)

In [27]:
data.drop('Timestamp', axis=1, inplace=True)

In [18]:
data

,year,month,day,hour,trips
Timestamp,,,,,
2018-01-01 00:00:00,2018,1,1,0,16714
2018-01-01 01:00:00,2018,1,1,1,19041
2018-01-01 02:00:00,2018,1,1,2,16590
2018-01-01 03:00:00,2018,1,1,3,12626
2018-01-01 04:00:00,2018,1,1,4,8739
...,...,...,...,...,...
2018-12-31 19:00:00,2018,12,31,19,14876
2018-12-31 20:00:00,2018,12,31,20,14434
2018-12-31 21:00:00,2018,12,31,21,14116


In [39]:
# Generate x and y matrices
eqn = """trips ~ -1 + hour + day +
      month"""
y, x = pt.dmatrices(eqn, data=data)

In [53]:
# Initialize and fit the model
model = LinearGAM(s(0) + s(1) + s(2))
modelFit = model.gridsearch(np.asarray(x), y)

100% (11 of 11) |########################| Elapsed Time: 0:00:04 Time:  0:00:04


In [49]:
titles = ['hour', 'day', 'month']

fig = subplots.make_subplots(rows=1, cols=3,
	subplot_titles=titles)
fig['layout'].update(height=400, width=1200,
	title='pyGAM', showlegend=False)

Layout({
    'annotations': [{'font': {'size': 16},
                     'showarrow': False,
                     'text': 'hour',
                     'x': 0.14444444444444446,
                     'xanchor': 'center',
                     'xref': 'paper',
                     'y': 1.0,
                     'yanchor': 'bottom',
                     'yref': 'paper'},
                    {'font': {'size': 16},
                     'showarrow': False,
                     'text': 'day',
                     'x': 0.5,
                     'xanchor': 'center',
                     'xref': 'paper',
                     'y': 1.0,
                     'yanchor': 'bottom',
                     'yref': 'paper'},
                    {'font': {'size': 16},
                     'showarrow': False,
                     'text': 'month',
                     'x': 0.8555555555555556,
                     'xanchor': 'center',
                     'xref': 'paper',
                     'y': 1.0,
         

In [50]:
for i, title in enumerate(titles):
  XX = model.generate_X_grid(term=i)
  pdep, confi = model.partial_dependence(term=i, width=.95)
  trace = go.Scatter(x=XX[:,i], y=pdep, mode='lines',
  	name='Effect')
  ci1 = go.Scatter(x = XX[:,i], y=confi[:,0],
  	line=dict(dash='dash', color='grey'),
    	name='95% CI')
  ci2 = go.Scatter(x = XX[:,i], y=confi[:,1],
  	line=dict(dash='dash', color='grey'),
    name='95% CI')
  if i<3:
    fig.append_trace(trace, 1, i+1)
    fig.append_trace(ci1, 1, i+1)
    fig.append_trace(ci2, 1, i+1)
  else:
    fig.append_trace(trace, 2, i-2)
    fig.append_trace(ci1, 2, i-2)
    fig.append_trace(ci2, 2, i-2)

In [51]:
fig

In [117]:
dataNew = pd.DataFrame([[j, i, 1] for i in range(1, 32) for j in range(0, 24)], columns=['hour', 'day', 'month'])

In [118]:
dataNew

,hour,day,month
0,0,1,1
1,1,1,1
2,2,1,1
3,3,1,1
4,4,1,1
...,...,...,...
739,19,31,1
740,20,31,1
741,21,31,1
742,22,31,1


In [119]:
xNew = pt.build_design_matrices([x.design_info], dataNew)

In [120]:
pred = modelFit.predict(xNew[0])

In [121]:
len(pred)

744